### Scraping Nigeria Rental data

This project scrapes the number of properties for rents updated on Nigeria Property Centre in Durumi within Federal capital Territory Abuja Nigeria

Note that it could be applied to any other area across nigeria by changing the name to reflect the area required

### STAGE ONE: PREPING THE ENVIRONMENT

Install Packages

In [ ]:
!pip install datatable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96.9 MB 106 kB/s 


Import Packages

In [ ]:
import pandas as pd
import datatable as dt
import time
import datetime
from datetime import datetime

Generate date for managing time series data

In [ ]:
current_time = datetime.now()
now = current_time.strftime('%m/%d/%Y')

Read in our base map for managing the whole process

In [ ]:
df = dt.fread('/content/nga.prop.centre.xlsx')
dfx = df.to_pandas()
dfx.head()

,state,lga,path
0,abuja,abaji,abuja/abaji
1,abuja,apo,abuja/apo
2,abuja,asokoro-district,abuja/asokoro-district
3,abuja,bwari,abuja/bwari
4,abuja,central-area-phase-2,abuja/central-area-phase-2


Read in our base URL

In [ ]:
url = 'https://nigeriapropertycentre.com/for-rent/'
#url = 'https://nigeriapropertycentre.com/for-rent/abuja?page=1'
#url = 'https://nigeriapropertycentre.com/for-rent/abuja/durumi'
#url = 'https://www.propertypro.ng/property-for-rent/in/abuja/durumi?search=&auto=&type=&bedroom=&min_price=&max_price='

Create indivual url for scraping stats for each state and local government

In [ ]:
page_no = '?page=1'
spath = dfx.iloc[1,2]
con_path = url + spath + page_no
con_path

'https://www.propertypro.ng/property-for-rent/in/abuja/durumi?search=&auto=&type=&bedroom=&min_price=&max_price=abuja/apo?page=1'

## STAGE TWO: BASIC DATA EXPLORATION

Read in each dable feom the site into pandas

In [ ]:
dfs = pd.read_html(con_path)

Select the first table from the above pandas tree

In [ ]:
for dw in dfs:
    if len(dw) == 5:
        table_1 = dw
        break
table_1       

,Property Type,Count
0,Flats,28
1,Houses,42
2,Land,-
3,Commercial Property,1
4,Event Centres / Venues,1


Select the second table from the above pandas tree

In [ ]:
for dw in dfs:
    if len(dw) == 2:
        table_2 = dw
        break
table_2 

,Type,1 bed,2 bed,3 bed,4 bed,5 bed
0,Flats,3,8,12,2,-
1,Houses,-,1,5,17,15


Select the third table from the above pandas tree

In [ ]:
for dw in dfs:
    if len(dw) == 6:
        table_3 = dw
        break
table_3

,Land Type,Count
0,Residential Land,-
1,Commercial Land,-
2,Industrial Land,-
3,Mixed-use Land,-
4,Other Land,-
5,All Land,-


## STAGE THREE: DATA MANIPULATION AND CLEANING 

## Table One(1)

Rename the table_1 coulumn to reflect rose readable syntax and standard

In [ ]:
b = dfx.iloc[0,0] + "." + dfx.iloc[0,1]
 # c = dfx.iloc[i,0] + "." + b
table_1.rename(columns={'Count':b, 'Property Type':'property.type'}, inplace = True)
table_1

,property.type,abuja.abaji
0,Flats,28
1,Houses,42
2,Land,-
3,Commercial Property,1
4,Event Centres / Venues,1


### Merge all local tables from table_1 to form a single table

In [ ]:
dd = table_1.copy() #
#print(dd)
for i in range(1, len(dfx)): #
  spath = dfx.iloc[i,2]
  con_path = url + spath
  dfs = pd.read_html(con_path)
  #print(dfs)
  b = dfx.iloc[i,0] + "." + dfx.iloc[i,1]
  for dw in dfs:
    if len(dw) == 5:
      dw.rename(columns={'Count':b, 'Property Type':'property.type'}, inplace = True)
      
      table = dw
      
      col = table.pop(b)
      #
      cue = i+1
      dd.insert(cue, b, col)
      #break   

col_names = dd.columns 
col_names = [sub.replace('-', '.') for sub in col_names]
dd.columns = col_names
dd

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


,property.type,abuja.abaji,abuja.apo,abuja.asokoro.district,abuja.bwari,abuja.central.area.phase.2,abuja.central.business.district,abuja.cultural.zones,abuja.dakibiyu,abuja.dakwo,...,rivers.obio.akpor,rivers.ogba.egbema.ndoni,rivers.ogu.bolo,rivers.okrika,rivers.omuma,rivers.opobo.nkoro,rivers.oyigbo,rivers.port.harcourt,rivers.tai,rivers.rivers
0,Flats,28,28,47,7,-,-,1,3,8,...,1,-,-,-,-,-,-,95,-,96
1,Houses,42,42,133,4,-,-,-,1,8,...,-,-,-,-,-,-,-,55,-,55
2,Land,-,-,1,-,-,-,-,-,-,...,-,-,-,-,-,-,-,2,-,2
3,Commercial Property,1,1,4,1,3,11,-,-,-,...,-,-,-,-,-,-,-,30,-,30
4,Event Centres / Venues,1,1,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,2,-,2


Add date column to the dataframe

In [ ]:
my_date = []
for i in range(len(dd)):
  my_date.append(now * 1)
dd.insert(0, 'date', my_date)  
dd

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


,date,property.type,abuja.abaji,abuja.apo,abuja.asokoro.district,abuja.bwari,abuja.central.area.phase.2,abuja.central.business.district,abuja.cultural.zones,abuja.dakibiyu,...,rivers.obio.akpor,rivers.ogba.egbema.ndoni,rivers.ogu.bolo,rivers.okrika,rivers.omuma,rivers.opobo.nkoro,rivers.oyigbo,rivers.port.harcourt,rivers.tai,rivers.rivers
0,10/28/2022,Flats,28,28,47,7,-,-,1,3,...,1,-,-,-,-,-,-,95,-,96
1,10/28/2022,Houses,42,42,133,4,-,-,-,1,...,-,-,-,-,-,-,-,55,-,55
2,10/28/2022,Land,-,-,1,-,-,-,-,-,...,-,-,-,-,-,-,-,2,-,2
3,10/28/2022,Commercial Property,1,1,4,1,3,11,-,-,...,-,-,-,-,-,-,-,30,-,30
4,10/28/2022,Event Centres / Venues,1,1,-,-,-,-,-,-,...,-,-,-,-,-,-,-,2,-,2


In [ ]:
dd.to_csv('for_rent.lag.abj.rivers01.csv')

## Table Two(2)

Rename the table_2 coulumn to reflect rose readable syntax and standard

In [ ]:
b = dfx.iloc[0,0] + "." + dfx.iloc[0,1]

col_names = table_2.columns 
col_names = [sub.replace(' ', '.') for sub in col_names]
col_names = [sub.replace('d', 'd.'+b) for sub in col_names]
col_names = [item.lower() for item in col_names]
table_2.columns = col_names
table_2

,type,1.bed.abuja.abaji,2.bed.abuja.abaji,3.bed.abuja.abaji,4.bed.abuja.abaji,5.bed.abuja.abaji
0,Flats,3,8,12,2,-
1,Houses,-,1,5,17,15


Merge all local tables from table_2 to form a single table

In [ ]:
de = table_2.copy() #
#print(dd)
for i in range(1, len(dfx)): #
  spath = dfx.iloc[i,2]
  con_path = url + spath
  dfs = pd.read_html(con_path)
  #print(dfs)
  b = dfx.iloc[i,0] + "." + dfx.iloc[i,1]
  for dw in dfs:
    if len(dw) == 2:
      col_names = dw.columns 
      col_names = [sub.replace(' ', '.') for sub in col_names]
      col_names = [sub.replace('d', 'd.'+b) for sub in col_names]
      col_names = [sub.replace('-', '.') for sub in col_names]
      col_names = [item.lower() for item in col_names]
      dw.columns = col_names
      table = dw.drop(['type'], axis=1)
      de = pd.concat([de, table], axis=1)
      
de

,type,1.bed.abuja.abaji,2.bed.abuja.abaji,3.bed.abuja.abaji,4.bed.abuja.abaji,5.bed.abuja.abaji,1.bed.abuja.apo,2.bed.abuja.apo,3.bed.abuja.apo,4.bed.abuja.apo,...,1.bed.rivers.tai,2.bed.rivers.tai,3.bed.rivers.tai,4.bed.rivers.tai,5.bed.rivers.tai,1.bed.rivers.rivers,2.bed.rivers.rivers,3.bed.rivers.rivers,4.bed.rivers.rivers,5.bed.rivers.rivers
0,Flats,3,8,12,2,-,3,8,12,2,...,-,-,-,-,-,10,63,20,-,-
1,Houses,-,1,5,17,15,-,1,5,17,...,-,-,-,-,-,-,5,9,30,8


### Add date column to the dataframe

In [ ]:
my_date = []
for i in range(len(de)):
  my_date.append(now * 1)
de.insert(0, 'date', my_date)  
de

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


,date,type,1.bed.abuja.abaji,2.bed.abuja.abaji,3.bed.abuja.abaji,4.bed.abuja.abaji,5.bed.abuja.abaji,1.bed.abuja.apo,2.bed.abuja.apo,3.bed.abuja.apo,...,1.bed.rivers.tai,2.bed.rivers.tai,3.bed.rivers.tai,4.bed.rivers.tai,5.bed.rivers.tai,1.bed.rivers.rivers,2.bed.rivers.rivers,3.bed.rivers.rivers,4.bed.rivers.rivers,5.bed.rivers.rivers
0,10/28/2022,Flats,3,8,12,2,-,3,8,12,...,-,-,-,-,-,10,63,20,-,-
1,10/28/2022,Houses,-,1,5,17,15,-,1,5,...,-,-,-,-,-,-,5,9,30,8


In [ ]:
de.to_csv('for_rent.lag.abj.rivers02.csv')

### Table Three (3)

Rename the table_3 column to reflect rose syntax and standard

In [ ]:
b = dfx.iloc[0,0] + "." + dfx.iloc[0,1]
table_3.rename(columns={'Count':b, 'Land Type':'land.type'}, inplace = True)
table_3

,land.type,abuja.abaji
0,Residential Land,-
1,Commercial Land,-
2,Industrial Land,-
3,Mixed-use Land,-
4,Other Land,-
5,All Land,-


### Merge all local tables from table_3 to form a single table

In [ ]:
df = table_3.copy() #
for i in range(1, len(dfx)): #
  spath = dfx.iloc[i,2]
  con_path = url + spath
  dfs = pd.read_html(con_path)
  #print(dfs)
  #b = dfx.iloc[i,1]
  b = dfx.iloc[i,0] + "." + dfx.iloc[i,1]
  for dw in dfs:
    if len(dw) == 6:
      dw.rename(columns={'Count':b, 'Land Type':'land.type'}, inplace = True)
      table = dw
      #
      col = table.pop(b)
      #
      cue = i+1
      df.insert(cue, b, col)
      #break   
col_names = df.columns
col_names = [sub.replace('-', '.') for sub in col_names]
df.columns = col_names
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


,land.type,abuja.abaji,abuja.apo,abuja.asokoro.district,abuja.bwari,abuja.central.area.phase.2,abuja.central.business.district,abuja.cultural.zones,abuja.dakibiyu,abuja.dakwo,...,rivers.obio.akpor,rivers.ogba.egbema.ndoni,rivers.ogu.bolo,rivers.okrika,rivers.omuma,rivers.opobo.nkoro,rivers.oyigbo,rivers.port.harcourt,rivers.tai,rivers.rivers
0,Residential Land,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,Commercial Land,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,Industrial Land,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,Mixed-use Land,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,2,-,2
4,Other Land,-,-,1,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
5,All Land,-,-,1,-,-,-,-,-,-,...,-,-,-,-,-,-,-,2,-,2


Add Date column to the dataframe

In [ ]:
my_date = []
for i in range(len(df)):
  my_date.append(now * 1)
df.insert(0, 'date', my_date)  
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


,date,land.type,abuja.abaji,abuja.apo,abuja.asokoro.district,abuja.bwari,abuja.central.area.phase.2,abuja.central.business.district,abuja.cultural.zones,abuja.dakibiyu,...,rivers.obio.akpor,rivers.ogba.egbema.ndoni,rivers.ogu.bolo,rivers.okrika,rivers.omuma,rivers.opobo.nkoro,rivers.oyigbo,rivers.port.harcourt,rivers.tai,rivers.rivers
0,10/28/2022,Residential Land,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,10/28/2022,Commercial Land,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,10/28/2022,Industrial Land,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,10/28/2022,Mixed-use Land,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,2,-,2
4,10/28/2022,Other Land,-,-,1,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
5,10/28/2022,All Land,-,-,1,-,-,-,-,-,...,-,-,-,-,-,-,-,2,-,2


convert our table to a csv file

In [ ]:
df.to_csv('for_rent.lag.abj.rivers03.csv')